In [52]:
import csv
import nltk
import re
import numpy as np
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, train_test_split
from scipy.sparse import hstack
from sklearn.metrics import make_scorer, accuracy_score, f1_score, precision_score, recall_score


DATA_PATH="../data/"

DATA = DATA_PATH + "labeled_data.csv"


In [57]:
def readData(path):
    data = []
    with open(path,'r') as file:
        data = [x for x in csv.reader(file, delimiter=',')]
    return data

def getTweets(raw):
    data = [x[6] for x in raw]
    return data

def getClass(raw):
    classes = [x[5] for x in raw]
    return np.array(classes)

def removePattern(tweet, pattern):
    r = re.findall(pattern, tweet)
    for x in r:
        tweet = re.sub(x, '', tweet)
    return tweet

def preprocess(data):
    cleanData = []
    for tweet in data:
        tweet = removePattern(tweet, "@[\w]*")
        tweet = tweet.replace("#", "") # Removing '#' from hashtags
        tweet = tweet.replace("[^a-zA-Z#]", " ") # Removing punctuation and special characters
        tweet = re.sub(r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+',"<URL>", tweet)
        tweet = re.sub(" +", " ", tweet)
        tweet = tweet.lower()
        cleanData.append(tweet)
        print(tweet)
    return cleanData

def tokenize(data):
    return TweetTokenizer.tokenize(data)

In [58]:
en_stopwords = set(stopwords.words("english")) 

wordVectorizer = CountVectorizer(
    analyzer = 'word',
    tokenizer = tokenize,
    lowercase = True,
    ngram_range=(1, 1),
    stop_words = en_stopwords)

charVectorizer = CountVectorizer(
    analyzer = 'char',
    tokenizer = tokenize,
    lowercase = True,
    ngram_range=(2, 6),
    stop_words = en_stopwords)


In [60]:
X = [x for x in tweets]
X = np.delete(np.array(X), [0])
y = np.delete(classes, [0])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
charVectorizer.fit(X)
train_char_features = charVectorizer.transform(X_train)
test_char_features = charVectorizer.transform(X_test)

# wordVectorizer.fit(X)
# train_word_features = wordVectorizer.transform(X_train)
# test_word_features = wordVectorizer.transform(X_test)


In [61]:
classifier = LogisticRegression(C=0.1, solver='sag')
classifier.fit(train_char_features, y_train)


/home/vaibhav/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/vaibhav/.local/lib/python3.6/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='sag',
          tol=0.0001, verbose=0, warm_start=False)

In [62]:
y_predict = classifier.predict(test_char_features)

In [63]:
def evaluate(target, predicted):
    f1 = f1_score(target, predicted, average='weighted')
    acc = accuracy_score(target, predicted)
    rec = recall_score(target, predicted, average = 'macro')
#     return f1, acc, rec
    print("f1 score: ", f1)
    print("avg recall", rec)    
    print("accuracy", acc)
evaluate(y_test, y_predict)

f1 score:  0.883456475192996
avg recall 0.6749308193652613
accuracy 0.8944926366754085


In [30]:
if __name__ == "__main__":
    raw = readData(DATA) 
    tweets = getTweets(raw)
    classes = getClass(raw)
    tweets = preprocess(tweets)


tweet
!!! rt : as a woman you shouldn't complain about cleaning up your house. &amp; as a man you should always take the trash out...
!!!!! rt : boy dats cold...tyga dwn bad for cuffin dat hoe in the 1st place!!
!!!!!!! rt dawg!!!! rt : you ever fuck a bitch and she start to cry? you be confused as shit
!!!!!!!!! rt : she look like a tranny
!!!!!!!!!!!!! rt : the shit you hear about me might be true or it might be faker than the bitch who told it to ya &57361;
!!!!!!!!!!!!!!!!!!": the shit just blows me..claim you so faithful and down for somebody but still fucking with hoes! &128514;&128514;&128514;"
!!!!!!": i can not just sit up and hate on another bitch .. i got too much shit going on!"
!!!!&8220;: cause i'm tired of you big bitches coming for us skinny girls!!&8221;
" &amp; you might not get ya bitch back &amp; thats that "
" :hobbies include: fighting mariam"bitch
" keeks is a bitch she curves everyone " lol i walked into a conversation like this. smh
" murda gang bitch its gang 

&8220;: what if i tell y'all i'm trying to turn up for bayou classic drinker &128520;&8221; bitch you got the id.
&8220;: &8220;: lol i 4got rose been dat nicca tho&8221; 1 never forget.&8221;lol yee i was trippin
&8220;: looking at nip rings i can't wait until i can change mine &128527;&8221; &128221;&128064;&128209; lmao
&8220;: &8220;: happy new year &127882;&127881;&127881;&127881;&127881;&127881;&127881;&127882;&127882;&8221; check yo clock hun we got another hour lol..&8221;they just did the lil count down thing
&8220;: bitch gotta work all hard to clean that shit without ripping her skin&8221; lmaosjdusubejs
&8220;: my friends are some insensitive bitches &128514;&8221;
&8220;: i'm tryna get some snow day dick &128514;&128514;&128514;&128514;&8221;gon make a nicca have a wreck out there tryn pull up on ya &128514;&128514;
&8220;: like why tf is this bitch following me ?&8221;&128064;
&8220;: cuz you a hoe lol&8221; cuz you went on your lil rant on me man!
&8220;: that wasn't me 

 he's a cheating wetback
 he called the kidd a pussy or something
 i couldn't find the regular one faggot
 
i likes yo page. i wants to make da lovin to a white girl wit a colored man watchin and pullin his weenis
 cdfu bitch you fried
 makes you a pussy. making stupid internet threats. you're a child and if it is about you which idk if it is, it's probably deserved.
 go buy them bitches
 shown* a tiny kitten you dumb fucking cunt
 
plot twist he is a faggot
 my brother. where tf you been at negro
 you ready to make this stats test our bitch?!
 i run cp in our community. local cops treat ppl like trash for that. i hate it. makes ppl think its a waste
 why are you showing yankees game in arizona when you promo washington huskies game? i bet would want my business
 some people do shit to get shot at and then be like "people hate me, i dodge bullets". karma is bitch.
 mane y'all niccas crazy af wiz old shit got niccas bitin his style n errthan
 chatt niccas love doin dat brah
 lol hell ya

 my mfn birthday!!! pink champaign in dat bitch!!!
 it sucks rachels sister is such a snobby bitch n rach is so nice!!!!!cant stand elissa, so snutty
 sup nappy headed hoe you got quiet
 starts making monkey faces behind you
 that pic with the red bone in braids and that tiger in the back for sale?
 teabaggers weakening food and environmental regulations is infinitely more scary and lethal than terrorism.
 looking at the royals and wondering what could have been if we didn't decimate the yankee farm system.
 shut up you nappy headed hoe
 happy birthday bitch&10084;&65039;&127881;
 
spiked that bitch
damned nailed her down
 you look like a monkey
 that area is a bitch regardless so i can imagine
 i ain't no lady bitch
 sheeze a yellow bone
 what that pussy do tho?
 lol ive gotten into with retards on here who actually think i am my avi -_-' ive gotten called ugly black bitch. like......
 it is, we be thugging on these bitches too hard now days
 bitches crazy &128553;&128514;
 i am worki

 bitch i'm gretchen weiners i do what i want
 wyd fag
 &128548;&128514;&128514;&128514;&128514; bitch
 he's a faggot
 nigger hello?
 that's true idk why your even in this convo you dumb nigger attention whore
 i can smell your fried chicken ass a mile away boi and clean your nasty ass ears ratchet bitch
 lmao you stupid nigger
 yuhh i'm finna hit that hoe! &128513;&128077;
 how many times did you listen to that pussy song today?
 some hoe shit
 you must have sent a bird call because i didn't get shit
 hell naw fuck them bitches serves them right
 really thou im looking like hulk hogan in this bitch
 but then they said "oh nah ur not" but dude i'm getting to be a heartless bitch have you not met me
 bitch u know what i mean spell champ these nuts
 yea i forgot i was talkin to a degenerate faggot
 your daughter can date this little fag bag <url>
 that hoe can't cook
 bitch stop hating on such an immaculate suit.
 my birthday hath already passed with little to no fanfare. so. enjoy your d

...

rt : i want someone i can call my own without another bitch staking claim on him simultaneously.
rt : got this bitch at work mad with me bc her ex bitch on my dick &128514;&128514;&128514;
rt : directioners, 1d's &amp; monkey around on set of stealmygirl video: <url> <url>
rt : ": ----&gt; tragedy smh rt : niggas don't know you but will throw dirt on your name for the pussy you h&8230;
rt : every bad bitch comes with mood swings
rt : you suppose to keep your hoes from finding out about each other
rt : yeah they trash. q... this man capitalized his h tho.... i'm takin all ya fans.
rt : liam gallagher came to see me perform tonight. he said i was a mental cunt. that's fucking amazing.
rt : bowebergdahl there hasn't been a trade this lopsided since babe ruth was traded to the yankees for "no no nanette" cash.
rt : ctfu, nigga's only call you a bitch when you speaking the truth about em or they don't get they way &128514;&128514;
rt : omg hahaha best thing you will watch today. milan lucic

rt : yelled out "come on bitch!" at myself, by myself, lifting weights as motivation...thanks guys, you always have my back.
rt : you shoulda learned how to bake, cuz bitches love cake
rt : meanwhile, in canada: a monkey in a winter coat escapes in an ikea <url> via wire
rt : you a bitch!
rt : too many faithful girls are single, and too many hoes are taken.&128581;
rt : wiz khalifas's song "black and yellow" is a beautiful song about a bruised banana
rt : people downloading time hop to be reminded they were faggots a year ago too
rt : yea but who gets more bitches grandma
rt : girls really made being a hoe kool on twitter
rt : i hate typing something so detailed &amp; somebody respond with "huh?" bitch i'm finna type the same thing twice dumb bitch
rt : ugly hoes be the loudest .. u need to duck off &amp; hide yourself frfr
rt : why all these hoes be wearin flowers around they head. i hope a bee sting they ass
rt : i'm a crazy bitch.. fml
rt : - these porn hoes moan at everything !!! &

rt : there might be a couple cotton pickers on the spurs but they all play unselfish white man basketball
rt : if a "man" tells you he doesn't eat pussy &amp;expects to have sex, just laugh. laugh as u put your clothes on, laugh as u g&8230;
rt : fake ass bitches bruh lol &128553;
rt : when you listening to a bitch complaining about a nigga that ain&8217;t even hers. <url>
rt : tf is a baby goal? bitch u gone have what you gone have. rt : baby goals &128525;&128525;&128525;&128525;&128076;&128175; <url>
rt : hate a bitch that run her mouth .
rt : rule 8- do pussy ass shit sometimes. like write her love notes. write long paragraphs about how you've never felt this way b&8230;
rt : is so wise yet so young. thank you for listening to me bitch &10084;&65039;
rt : waka flaka a lucky retard lol
rt : if i was stranded on an island and could only bring one thing, i would bring dora. that bitch has everything in that fu ...
rt : real friends don't get offended when you call them bitch, hoe or s

rt": i wonder how much pussy justin bieber gets....."
rt'in your own shit now pussy what a joke you'd end up just like all the rest no show chicken shit mother fucker
rt, if you agree!
northkorea says barackobama looks like;
"a monkey, who belongs in an african zoo" 
rofl! <url>
rt: thank you, captain. it'll never be the same. jeter2 <url>
raaaayciss bitch &8220;: michelle obama: african-americans should vote for democrats <url> via &8221;
rack city bitch !
raised round real niccas no decoy
rally's sound good af. but ion have no hoes so we can't take that ride &128530;
rambo bout to flex on these hoes
rams too ": lakers as a whole is trash rt : jordan hill is trash. play davis and randle more"
ran here from my house too... i'm kinda retarded. lol
rand went full retard some time ago &8220;: isn't "playing the game"... he's just retarded.
 <url>
rap bitches prolly scared of remy ma lol
rape a ho ben! lol
ratchet lol ass lol bitch
raunchy ass hoe.
raunchy ass to pussy action with sakurako

what they taught us, man them bad bitches only want the ballers, the starters.. we hoopin now the hoes wanna guard us..
what time jamboree start thursday. so i ca smile for the ugly hoes. &128514;&128128;
what u don't do another hoe will
what up fake ass hoe? i'm not gone fought you today. gimme my nigga hand back
what up twats? i mean twits... &128522;
what was that knob to get shit back in focus when it started getting fuzzy?
what would yall do?.....i would act like one of them nd act like we lookin for the same bitch lmao <url>
what you doing bitch suck it don't bite it
what you gonna do if she cheat ? cry ? fight for her ? no kick that bitch out and go straight to tootsies
what you know about getting money low key .. hanging with the street niccas fukn wit the og's
what you saying hoe?
what's a queen without a king?

...a lonely ass bitch!
what's a trash party ?
what's really funny bout this pic is i'm sure they believe them hoes is bad &128514;&128514;&128514;&128514; <url>
what's